In [ ]:
import os, sys
from tqdm import tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import zipfile
import time
import shutil
import collections
from pathlib import Path

In [ ]:
import warnings
warnings.filterwarnings('ignore')

### Loading packages

In [ ]:
import sys
from pathlib import Path

here_path = Path().resolve()
repo_path = here_path.parents[1]
sys.path.append(str(repo_path))

In [ ]:
from py.utils import verifyDir, verifyFile, verifyType

In [ ]:
from py.config import Config

cfg = Config()

np.random.seed(cfg.RANDOM_STATE)
cfg.DATA_PATH, cfg.MODEL_PATH

In [ ]:
QSCORE_PATH=f"{cfg.DATA_PATH}pp2/{cfg.SCORING_METHOD}/{cfg.PLACE_LEVEL}/"
IMAGES_PATH = f"{cfg.DATA_PATH}pp2/images/"

ADE20K_DIR = f"{cfg.DATA_PATH}{cfg.ADE20K_DATASET}/"
CITYSCAPES_DIR = f"{cfg.DATA_PATH}{cfg.CITYSCAPES_DATASET}/"

In [ ]:
verifyDir(CITYSCAPES_DIR)

#### Loading segmentations

In [ ]:
df = pd.read_csv(f"{ADE20K_DIR}ade20k_color_classes.csv", sep=";", low_memory=False)
df

In [ ]:
cityscape_data_dict = {"0": "road",
                    "1": "sidewalk",
                    "2": "building",
                    "3": "wall",
                    "4": "fence",
                    "5": "pole",
                    "6": "traffic light",
                    "7": "traffic sign",
                    "8": "vegetation",
                    "9": "terrain",
                    "10": "sky",
                    "11": "person",
                    "12": "rider",
                    "13": "car",
                    "14": "truck",
                    "15": "bus",
                    "16": "train",
                    "17": "motorcycle",
                    "18": "bicycle"
                       }

cityscapes_df = pd.DataFrame(list(cityscape_data_dict.items()), columns=['class_id', 'class_name'])
cityscapes_df["main_class"] = cityscapes_df["class_name"].apply(lambda x: "_".join(x.split()))
cityscapes_df

In [ ]:
cde_df = pd.merge(cityscapes_df, df[["main_class", "RGB_color", "hex_color", "isthing"]].copy(), how="left", on="main_class")
cde_df

In [ ]:
cde_df.loc[(cde_df["main_class"]=="motorcycle"), "RGB_color"] = '(163, 0, 255)'
cde_df.loc[(cde_df["main_class"]=="motorcycle"), "hex_color"] = "#A300FF"
cde_df.loc[(cde_df["main_class"]=="motorcycle"), "isthing"] = 1

cde_df.loc[(cde_df["main_class"]=="vegetation"), "RGB_color"] = '(4, 200, 3)'
cde_df.loc[(cde_df["main_class"]=="vegetation"), "hex_color"] = "#04C803"
cde_df.loc[(cde_df["main_class"]=="vegetation"), "isthing"] = 0

cde_df.loc[(cde_df["main_class"]=="terrain"), "RGB_color"] = '(120, 120, 70)'
cde_df.loc[(cde_df["main_class"]=="terrain"), "hex_color"] = "#787846"
cde_df.loc[(cde_df["main_class"]=="terrain"), "isthing"] = 0

cde_df.loc[(cde_df["main_class"]=="traffic_light"), "RGB_color"] = '(41, 0, 255)'
cde_df.loc[(cde_df["main_class"]=="traffic_light"), "hex_color"] = "#2900FF"
cde_df.loc[(cde_df["main_class"]=="traffic_light"), "isthing"] = 1

cde_df.loc[(cde_df["main_class"]=="traffic_sign"), "RGB_color"] = '(255, 5, 153)'
cde_df.loc[(cde_df["main_class"]=="traffic_sign"), "hex_color"] = "#FF0599"
cde_df.loc[(cde_df["main_class"]=="traffic_sign"), "isthing"] = 1

cde_df.loc[(cde_df["main_class"]=="rider"), "RGB_color"] = '(204, 70, 3)'
cde_df.loc[(cde_df["main_class"]=="rider"), "hex_color"] = "#CC4603"
cde_df.loc[(cde_df["main_class"]=="rider"), "isthing"] = 1

cde_df.loc[(cde_df["main_class"]=="train"), "RGB_color"] = '(255, 6, 51)'
cde_df.loc[(cde_df["main_class"]=="train"), "hex_color"] = "#FF0633"
cde_df.loc[(cde_df["main_class"]=="train"), "isthing"] = 1

cde_df

In [ ]:
cde_df.to_csv(f"{CITYSCAPES_DIR}cityscapes_color_classes.csv", sep=";", index=False)